In [2]:
import pandas as pd
import json

In [3]:
rinput = ["resultId", "raceId", "driverId", "constructorId", "points"]
dinput = ["driverId", "driverRef", "code", "forename", "surname"]
drivers = pd.read_csv("drivers.csv", usecols=dinput)
results = pd.read_csv("results.csv", usecols=rinput)

In [4]:
teammates = results.groupby(["constructorId", "raceId"])["driverId"].apply(lambda x: pd.Series(x.tolist())).unstack().reset_index().iloc[:, 0:4]

In [5]:
teammates.rename(columns={0: "driverId", 1: "driver2"}, inplace=True)

In [6]:
df1 = teammates.groupby(["driverId"])["driver2"].agg(list).reset_index(name="dupe_partners")
df2 = teammates.groupby(["driver2"])["driverId"].agg(list).reset_index(name="dupe_partners")

In [7]:
df2.rename(columns={"driver2": "driverId"}, inplace=True)

In [8]:
li1 = df1["driverId"].to_list()
li2 = df2["driverId"].to_list()

In [9]:
s = set(li1)
missing = [x for x in li2 if x not in s]

In [10]:
missingrows = df2[df2.driverId.isin(missing)]

In [11]:
dupes = pd.concat([df1, missingrows]).reset_index(drop=True)

In [12]:
dupes["teammates_set"] = dupes["dupe_partners"].apply(set)
dupes["teammates"] = dupes["teammates_set"].apply(list)

In [13]:
teammates = dupes.drop(["dupe_partners", "teammates_set"], axis=1)

In [14]:
merged = pd.merge(drivers, teammates, on="driverId")

In [15]:
merged.to_csv("teammates.csv")

In [ ]:
# getting data into node, link format

In [16]:
merged["fullname"] = merged["forename"] + " " + merged["surname"]

In [17]:
ttszn = merged[merged['fullname'].isin(["Max Verstappen", "Charles Leclerc", "Sergio Pérez", "George Russell", "Carlos Sainz", "Lewis Hamilton", "Lando Norris", "Esteban Ocon", "Fernando Alonso", "Valtteri Bottas", "Daniel Ricciardo", "Sebastian Vettel", "Kevin Magnussen", "Pierre Gasly", "Lance Stroll", "Mick Schumacher", "Yuki Tsunoda", "Guanyu Zhou", "Alexander Albon", "Nicholas Latifi", "Nyck de Vries", "Nico Hülkenberg"])]

In [18]:
ttszn

,driverId,driverRef,code,forename,surname,teammates,fullname
0,1,hamilton,HAM,Lewis,Hamilton,"[3.0, 4.0, 5.0, 847.0, 18.0, 822.0]",Lewis Hamilton
3,4,alonso,ALO,Fernando,Alonso,"[1.0, 35.0, 58.0, 838.0, 839.0, 8.0, 12.0, 13....",Fernando Alonso
19,20,vettel,VET,Sebastian,Vettel,"[2.0, 7.0, 8.0, 840.0, 844.0, 17.0, 817.0, 24.0]",Sebastian Vettel
425,842,gasly,GAS,Pierre,Gasly,"[832.0, 843.0, 852.0, 826.0, 830.0]",Pierre Gasly
669,807,hulkenberg,HUL,Nico,Hülkenberg,"[832.0, 835.0, 840.0, 814.0, 815.0, 817.0, 821...",Nico Hülkenberg
676,815,perez,PER,Sergio,Pérez,"[839.0, 807.0, 840.0, 18.0, 155.0, 830.0]",Sergio Pérez
678,817,ricciardo,RIC,Daniel,Ricciardo,"[839.0, 807.0, 846.0, 818.0, 20.0, 24.0, 826.0...",Daniel Ricciardo
683,822,bottas,BOT,Valtteri,Bottas,"[1.0, 13.0, 813.0, 847.0, 855.0]",Valtteri Bottas
686,825,kevin_magnussen,MAG,Kevin,Magnussen,"[835.0, 18.0, 850.0, 854.0, 154.0]",Kevin Magnussen
691,830,max_verstappen,VER,Max,Verstappen,"[832.0, 842.0, 815.0, 848.0, 817.0]",Max Verstappen


In [19]:
forcedata = {"nodes": [], "links": []}
for idx, row in ttszn.iterrows():
    forcedata["nodes"].append({"id": row["driverId"], "name": row["fullname"]})

In [22]:
for dict_ in vals:
    key = dict_["id"]
    for val in dict_["tm"]:
        forcedata["links"].append({"source": key, "target": val["id"], "value": val["cnt"]})
        
forcedata

{'nodes': [{'id': 1, 'name': 'Lewis Hamilton'},
  {'id': 4, 'name': 'Fernando Alonso'},
  {'id': 20, 'name': 'Sebastian Vettel'},
  {'id': 842, 'name': 'Pierre Gasly'},
  {'id': 807, 'name': 'Nico Hülkenberg'},
  {'id': 815, 'name': 'Sergio Pérez'},
  {'id': 817, 'name': 'Daniel Ricciardo'},
  {'id': 822, 'name': 'Valtteri Bottas'},
  {'id': 825, 'name': 'Kevin Magnussen'},
  {'id': 830, 'name': 'Max Verstappen'},
  {'id': 832, 'name': 'Carlos Sainz'},
  {'id': 839, 'name': 'Esteban Ocon'},
  {'id': 840, 'name': 'Lance Stroll'},
  {'id': 844, 'name': 'Charles Leclerc'},
  {'id': 846, 'name': 'Lando Norris'},
  {'id': 847, 'name': 'George Russell'},
  {'id': 848, 'name': 'Alexander Albon'},
  {'id': 849, 'name': 'Nicholas Latifi'},
  {'id': 852, 'name': 'Yuki Tsunoda'},
  {'id': 854, 'name': 'Mick Schumacher'},
  {'id': 855, 'name': 'Guanyu Zhou'},
  {'id': 856, 'name': 'Nyck de Vries'}],
 'links': [{'source': 1, 'target': 4, 'value': 17},
  {'source': 1, 'target': 847, 'value': 29},
  

In [21]:
vals = [
    {"id": 1, "tm": [{"id": 4, "cnt": 17}, {"id": 847, "cnt": 29}]}, 
    {"id": 4, "tm": [{"id": 1, "cnt": 17}, {"id": 839, "cnt": 44}]}, 
    {"id": 20, "tm": [{"id": 817, "cnt": 19}, {"id": 840, "cnt": 43}, {"id": 844, "cnt": 38}]}, 
    {"id": 842, "tm": [{"id": 830, "cnt": 12}, {"id": 832, "cnt": 2}, {"id": 852, "cnt": 44}]}, 
    {"id": 807, "tm": [{"id": 815, "cnt": 61}, {"id": 817, "cnt": 21}, {"id": 832, "cnt": 25}, {"id": 840, "cnt": 5}]}, 
    {"id": 815, "tm": [{"id": 807, "cnt": 61}, {"id": 830, "cnt": 51}, {"id": 839, "cnt": 41}, {"id": 840, "cnt": 37}]}, 
    {"id": 817, "tm": [{"id": 20, "cnt": 19}, {"id": 807, "cnt": 21}, {"id": 830, "cnt": 58}, {"id": 839, "cnt": 17}, {"id": 846, "cnt": 44}]},
    {"id": 822, "tm": [{"id": 1, "cnt": 100}, {"id": 847, "cnt": 1}, {"id": 855, "cnt": 29}]},
    {"id": 825, "tm": [{"id": 854, "cnt": 23}]}, 
    {"id": 830, "tm": [{"id": 842, "cnt": 12}, {"id": 815, "cnt": 51}, {"id": 817, "cnt": 58}, {"id": 832, "cnt": 23}, {"id": 848, "cnt": 26}]}, 
    {"id": 832, "tm": [{"id": 807, "cnt": 25}, {"id": 830, "cnt": 23}, {"id": 844, "cnt": 51}, {"id": 846, "cnt": 38}]}, 
    {"id": 839, "tm": [{"id": 4, "cnt": 44}, {"id": 815, "cnt": 41}, {"id": 817, "cnt": 17}]}, 
    {"id": 840, "tm": [{"id": 20, "cnt": 43}, {"id": 807, "cnt": 5}, {"id": 815, "cnt": 37}]}, 
    {"id": 844, "tm": [{"id": 20, "cnt": 38}, {"id": 832, "cnt": 38}]}, 
    {"id": 846, "tm": [{"id": 817, "cnt": 44}, {"id": 832, "cnt": 38}]}, 
    {"id": 847, "tm": [{"id": 1, "cnt": 29}, {"id": 849, "cnt": 44}]}, 
    {"id": 848, "tm": [{"id": 830, "cnt": 26}, {"id": 849, "cnt": 22}]}, 
    {"id": 849, "tm": [{"id": 847, "cnt": 44}, {"id": 848, "cnt": 22}]}, 
    {"id": 852, "tm": [{"id": 842, "cnt": 44}]}, 
    {"id": 854, "tm": [{"id": 825, "cnt": 23}]}, 
    {"id": 855, "tm": [{"id": 822, "cnt": 29}]}, 
    {"id": 856, "tm": [{"id": 849, "cnt": 2}]},
       ]

In [149]:
with open("teammates.json", "w") as outfile:
    json.dump(forcedata, outfile)

In [79]:
rinput = ["raceId", "date", "year", "name"]
csinput = ["constructorId", "raceId", "position"]
cinput = ["constructorId", "name"]
races = pd.read_csv("races.csv", usecols=rinput)
const_standings = pd.read_csv("constructor_standings.csv", usecols=csinput)
const = standings = pd.read_csv("constructors.csv", usecols=cinput)

In [80]:
ccsmerged = const_standings.merge(const, how='left', on="constructorId")

In [81]:
racesmerged = ccsmerged.merge(races, how='left', on="raceId")

In [91]:
racesmerged = racesmerged[racesmerged["year"] == 2022]
racesmerged.rename(columns={"name_x": "constructor", "name_y": "circuit"}, inplace=True)
racesmerged = racesmerged[["position", "constructor", "circuit", "date"]]

In [102]:
racesmerged

,position,constructor,circuit,date
12706,10,Red Bull,Bahrain Grand Prix,2022-03-20
12707,9,McLaren,Bahrain Grand Prix,2022-03-20
12708,8,Williams,Bahrain Grand Prix,2022-03-20
12709,7,Aston Martin,Bahrain Grand Prix,2022-03-20
12710,6,AlphaTauri,Bahrain Grand Prix,2022-03-20
...,...,...,...,...
12926,9,AlphaTauri,Abu Dhabi Grand Prix,2022-11-20
12927,7,Aston Martin,Abu Dhabi Grand Prix,2022-11-20
12928,10,Williams,Abu Dhabi Grand Prix,2022-11-20
12929,5,McLaren,Abu Dhabi Grand Prix,2022-11-20


In [101]:
racesmerged.to_csv("const.csv")

In [100]:
racesmerged.groupby(["constructor"])["position"].agg(list)

constructor
Alfa Romeo        [4, 6, 6, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, ...
AlphaTauri        [6, 7, 8, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, ...
Alpine F1 Team    [5, 4, 5, 6, 6, 6, 6, 5, 5, 5, 5, 4, 4, 4, 4, ...
Aston Martin      [7, 9, 10, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9,...
Ferrari           [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...
Haas F1 Team      [3, 5, 7, 8, 8, 8, 8, 8, 9, 8, 7, 7, 7, 7, 7, ...
McLaren           [9, 8, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, ...
Mercedes          [2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ...
Red Bull          [10, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...
Williams          [8, 10, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10,...
Name: position, dtype: object